In [1]:
#Colab setting
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/RecommanderSystems')

import os
os.chdir('/content/drive/My Drive/RecommanderSystems')

Mounted at /content/drive


In [5]:
from types import SimpleNamespace

config = SimpleNamespace(
    seed = 1,
    data = "Steam.txt", # Beauty.txt / ml-1m.txt
    dropout = 0.5, # 0.2 for ml-1m
    n = 50, # 300 for ml-1m
    d = 40,
    batch_size = 128,
    test_batch_size = 100,
    weight_decay = 5e-4,
    lr = 1e-3,
    epoch = 200,
    patience = 20

)

## Dataset

SASRec의 경우 학습을 시킬 때 user 당 item의 시퀀스가 필요

In [11]:
from collections import defaultdict

user_item_sequence_dict = defaultdict(list)

u_max = -1
i_max = -1

data = open(config.data, 'r')
for line in data:
  u, i = line.strip().split(' ') # (user, item) 조합의 txt 파일 읽기
  u = int(u)
  i = int(i)
  user_item_sequence_dict[u].append(i)
  if u > u_max:
    u_max = u
  if i > i_max:
    i_max = i

user_train = {}
user_valid = {}
user_test = {}

for u in user_item_sequence_dict:
  seq = user_item_sequence_dict[u]
  l = len(seq)
  user_valid[u] = []
  user_test[u] = []
  if l < 3:
    padded_seq = [0] * (config.n - l) + seq
    user_train[u] = padded_seq

  elif l < config.n:
    # padding 필요
    padded_seq = [0] * (config.n - l) + seq
    user_train[u] = padded_seq[:-2]
    user_valid[u] = padded_seq[-2] # n - 1번째
    user_test[u] = padded_seq[-1] # n번째
  else:
    # l > config.n
    recent_seq = seq[-config.n:]
    user_train[u] = recent_seq[:-2]
    user_valid[u] = recent_seq[-2]
    user_test[u] = recent_seq[-1]

In [ ]:
import torch
import torch.nn as nn

class SASRec(nn.Module):
    def __init__(self, config, user_num = u_max, item_num = i_max):
        super(SASRec, self).__init__()

        self.user_num = user_num
        self.item_num = item_num

        # M : item embedding (N X d) 인데 item index는 1 부터 시작 => + 1해주기
        self.M = nn.Prameter(torch.rand(self.item_num + 1, config.d) *1e-2)
